In [ ]:
import librosa as rs
import numpy as np
from scipy import signal


In [ ]:

path_noise = "/home/thk/SE/noise/rec_4ch_train_1.wav"
noise = rs.load(path_noise,sr=16000,mono=False)[0]
path_clean = "/home/DB/DNS-Challenge-16kHz/datasets_fullband/clean_fullband/read_speech/book_07500_chp_0001_reader_09713_48_seg_0.wav"
clean = rs.load(path_clean,sr=16000)[0]
path_rir = "/home/thk/SE/rir/RIR_1m_30deg_1.wav"
rir = rs.load(path_rir,sr=16000,mono=False)[0]

In [ ]:
len_data = 49152
def match_length(wav,idx_start=None):
    if max(wav.shape) > len_data : 
        left = max(wav.shape) - len_data
        if idx_start is None :
            idx_start = np.random.randint(left)
        if len(wav.shape) > 1:
            wav = wav[:,idx_start:idx_start+len_data]
        else:
            wav = wav[idx_start:idx_start+len_data]
    elif max(wav.shape) < len_data : 
        shortage = len_data - max(wav.shape) 
        if len(wav.shape) > 1:
            wav = np.pad(wav,((0,0),(0,shortage)))
        else :
            wav = np.pad(wav,(0,shortage))
    return wav, idx_start

In [ ]:
clean,_ = match_length(clean)
noise,_ = match_length(noise)

In [ ]:
'''
clean : single channel, rir & noise = 3ch => noisy = 3ch
make code <clean = signal.fftconvolve(clean, rir)[:len(clean)]>
to work with 3ch
so, make clean to 3ch using rir data with fftconvolve
'''
# dummy clean shape like noise
clean_ = np.repeat(signal.fftconvolve(clean, rir[0,:])[:,np.newaxis],3,axis=1).T
for i in range(3):
    clean_[i,:] = signal.fftconvolve(clean, rir[i,:])


In [ ]:
# save clean_ as rir_mix_clean.wav with soundfile
import soundfile as sf
sf.write("clean.wav",clean.T,16000)
sf.write("rir_mix_clean.wav",clean_.T,16000)
print(clean_.shape,clean.shape,rir.shape,clean_.shape[1]-clean.shape[0])

In [ ]:
clean.shape

In [ ]:
def norm_amplitude(y, scalar=None, eps=1e-6):
        if not scalar:
            scalar = np.max(np.abs(y)) + eps

        return y / scalar, scalar
def tailor_dB_FS(y, target_dB_FS=-25, eps=1e-6):
        rms = np.sqrt(np.mean(y ** 2))
        scalar = 10 ** (target_dB_FS / 20) / (rms + eps)
        y *= scalar
        return y, rms, scalar
print(clean)
clean, _ = norm_amplitude(clean)
print(clean)
clean, _, _ = tailor_dB_FS(clean)
print(clean)
clean_rms = (clean ** 2).mean() ** 0.5
print(clean_rms)

In [ ]:
print(noise)
noise, _ = norm_amplitude(noise)
print(noise)
noise, _, _ = tailor_dB_FS(noise)
print(noise)
noise_rms = (noise ** 2).mean() ** 0.5
print(noise_rms)

In [ ]:
SNR = noisy_target_dB_FS = np.random.randint(
            -12,-7
        )
snr_scalar = clean_rms / (10 ** (SNR / 20)) / (noise_rms + 1e-7)
noise *= snr_scalar
noisy = clean + noise
noisy

In [ ]:
noisy_target_dB_FS = np.random.randint(
            -35,
            -15
        )

# rescale noisy RMS
noisy, _, noisy_scalar = tailor_dB_FS(noisy, noisy_target_dB_FS)
clean *= noisy_scalar
def is_clipped(y, clipping_threshold=0.999):
        return any(np.abs(y) > clipping_threshold)

In [ ]:
def SNR(estim,target, requires_grad=False,device="cuda:0") :
    if estim.shape != target.shape : 
        raise Exception("ERROR::metric.py::SIR:: output shape != target shape | {} != {}".format(estim.shape,target.shape))
    estim = torch.Tensor(estim)
    target = torch.Tensor(target)

    s_target = (torch.inner(estim,target)*target/torch.inner(target,target))

    tmp = estim - s_target 
    e_noise = (tmp)

    SNR = (torch.inner(s_target,s_target))/torch.inner(e_noise,e_noise)
    return 10*torch.log10(SNR)